<div style='background-image: url("../../share/images/header.svg") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Computational Seismology</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">Parallel implementation for the acoustic wave equation</div>
        </div>
    </div>
</div>

Seismo-Live: http://seismo-live.org

##### Authors:
* David Vargas ([@dvargas](https://github.com/davofis))
* Heiner Igel ([@heinerigel](https://github.com/heinerigel))

This notebook covers the following aspects:

* Initial configuration of setup describing input parameters for the simulation 
* Domain partitioning for parallel implementation
* Local communication between neighboring cell, ghost cells
* Finite differences solution of the 1D acoustic wave equation
* Visualization of the acoustic wave field in space and time
* Animated wave field

### Numerical Solution (Finite Differences Method)

The acoustic wave equation in 1D with constant density is given by 

$$
\partial^2_t p(x,t) \ = \ c(x)^2 \partial_x^2 p(x,t) + s(x,t)
$$

where $p$ represents the pressure field, $c$ acoustic velocity, and $s$ the source term. Second derivatives of the pressure field with respect time and space can be approximated with a difference formula such as

$$
\partial^2_t p(x,t) \ \approx \ \frac{p(x,t+dt) - 2 p(x,t) + p(x,t-dt)}{dt^2} 
$$

similarly, we define and equivalent formula for the space derivative. Next, we introduce these approximations into the wave equation, this allows us to formulate the pressure p(x) for the time step $t+dt$ in the future as a function of the pressure at present time $t$ and $t-dt$ in the past. Additionally, it is convenient to use upper and lower index notation. The upper index indicates time and the lower space. Then

$$
 \frac{p_{i}^{n+1} - 2 p_{i}^n + p_{i}^{n-1}}{\mathrm{d}t^2} \ = \ c^2 ( \partial_x^2 p) \ + s_{i}^n
$$

solving for $p_{i}^{n+1}$.

The explicit extrapolation scheme can be written as 

$$
p_{i}^{n+1} \ = \ c_i^2 \mathrm{d}t^2 \left[ \partial_x^2 p \right]
+ 2p_{i}^n - p_{i}^{n-1} + \mathrm{d}t^2 s_{i}^n
$$

and the space derivatives are determined by 

$$
\partial_x^2 p \ = \ \frac{p_{i+1}^{n} - 2 p_{i}^n + p_{i-1}^{n}}{\mathrm{d}x^2}
$$

### Getting started

Before you start, make sure you have launched a new Ipython cluster with the desired number of engines. Having done that, run the Ipython setup cell as well as the imports cell, this will allow you to use MPI4Py into the jupyter notebook. The next cells are dedicated to the numerical implementation itself. First, the initialization of all parameters is done, you can modify those parameters to evaluate the performance and scalability of the code. Second, an independent cell is dedicated to perform time extrapolation, it is here where all parallel computations take place. Finally, visualization is implemented in the last part of the notebook, you will find an animated plot of the wave field as well as an image of the field itself in space-time  


### Ipython cluster setup

In [ ]:
# Import Libraries (PLEASE RUN THIS CODE FIRST!) 
# ----------------------------------------------
from ipyparallel import Client
cluster = Client(profile='mpi')
cluster.block = True  # use synchronous computations
dview = cluster[:]
dview.activate()      # enable magics

cluster.ids

### Imports
Libraries are imported on all workers. This is a configuration step for the exercise. Please run it before the simulation code!

In [ ]:
%%px
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpi4py import MPI

import warnings
warnings.filterwarnings("ignore")
#==================================
comm = MPI.COMM_WORLD

size = comm.Get_size()
rank = comm.Get_rank()
name = MPI.Get_processor_name()
#==================================

### 1. Initialization of setup
This cell initializes all necessary variables, arrays, and sets the initial conditions.

In [ ]:
%%px
# Parameter Configuration
#---------------------------------------------------------------
nx   = 500         # number of grid points in x-direction
dx   = 1.8         # grid point distance in x-direction
nt   = 1500        # maximum number of time steps
dt   = 0.005       # time step
c    = 333.        # wave speed in medium (m/s)
isrc = 250         # source location in grid in x-direction
iplot = 10         # Snapshot frequency

# CFL Stability Criterion
#---------------------------------------------------------------
eps  = c * dt / dx    # epsilon value
if rank == 0: print('Stability criterion =', eps)
  
nx_loc = int(nx/size)    # x-grid points per process

#%% Source Time Function (Gaussian)
#---------------------------------------------------------------
f0   = 25.       # dominant frequency of the source (Hz)
t0   = 2. / f0   # source time shift
if rank == 0: print('Source frequency =', f0, 'Hz')
 
src  = np.zeros(nt + 1)
t = np.linspace(0 * dt, nt * dt, nt)
# 1st derivative of a Gaussian
src  = -2. * (t - t0) * (f0**2) * (np.exp(-1.0 * (f0**2) * (t - t0)**2))

# Initialize pressure fields
#---------------------------------------------------------------
p    = np.zeros(nx) # p at time n (now)
pold = np.zeros(nx) # p at time n-1 (past)
pnew = np.zeros(nx) # p at time n+1 (present)
d2px = np.zeros(nx) # 2nd space derivative of p

# Initialize local vectors
#---------------------------------------------------------------
d2px_loc = np.zeros(nx_loc+2)   # add ghost cells in the borders
p_loc    = np.zeros(nx_loc+2)   # add ghost cells in the borders

# Initialize space coordinate
#--------------------------------------------------------------- 
x = np.arange(nx) * dx

### 2. Finite Differences solution 

We implement the finite difference solution using the extrapolation scheme:

\begin{equation}
p_{i}^{n+1} \ = \ c_i^2 \frac{\mathrm{d}t^2}{ \mathrm{d}x^2}\left[ p_{i+1}^{n} - 2 p_{i}^n + p_{i-1}^{n} \right] + 2p_{i}^n - p_{i}^{n-1} + \mathrm{d}t^2 s_{i}^n
\end{equation}

---
<div style="text-align: justify">

<p style="width:50%;float:right;padding-left:60px;padding-right:40px">
<img src=images/ghost.png>
<span style="font-size:smaller">
</span>
</p>
Since the explicit extrapolation scheme involve data dependence in time, i.e. the future amplitude value depends on the present and past states, this part can no be easily parallelized. However, we can parallelize the space derivatives by domain decomposition. Basically, the space (Mesh) is partitioned among all processors, they will perform derivatives locally and interact with each other via local communication. The last part is crucial so please focus your attention into the lines 'Domain partitioning with scatter', 'Local communicators' and 'Master gathers local derivatives'. The figure on the right hand side illustrates the basic idea behind local ghost cells 
</div>

### High-order operators
As an exercise, you may extend the code to higher order by adding a 5-point difference operator. The 5-pt weights are: 
$
[-1/12, 4/3, -5/2, 4/3, -1/12] / dx^2
$. 
Compare simulations of the 3-point and 5-point operators.


In [ ]:
%%px
#---------------------------------------------------------------
# TIME EXTRAPOLATION 
#--------------------------------------------------------------- 
P_xt = []    # Define array to store wavefield in space-time p(x,t)

comm.Barrier()         # Synchronize processors 
t_start = MPI.Wtime()  # start MPI timer

for it in range(nt):
    # Domain partitioning with scatter, local fields on workers
    comm.Scatter( [p, MPI.DOUBLE], [p_loc[1:nx_loc+1], MPI.DOUBLE] )
    # --------------------------------------------------------
    # Local communicators
    # --------------------------------------------------------
    # Send p[0] from ID's+1 to ID's
    if 0 < rank:
        comm.send(p_loc[1], dest=rank-1, tag=1)
    # Receive p[x_loc] in ID's from to ID+1
    if rank < size-1:
        p_loc[nx_loc+1] = comm.recv(source=rank+1, tag=1)
    # Send p[nx_loc] from ID's to ID's+1
    if rank < size-1:
        comm.send(p_loc[nx_loc], dest=rank+1, tag=2)
    # Receive p[x_loc] in ID's from to ID+1
    if 0 < rank:
        p_loc[0] = comm.recv(source=rank-1, tag=2)     
        
    # --------------------------------------------------------
    # Space derivative, 3 point operator FD scheme
    # --------------------------------------------------------
    for i in range(1, nx_loc+1):
        d2px_loc[i] = (p_loc[i + 1] - 2 * p_loc[i] + p_loc[i - 1]) / dx ** 2
    
    # Master gathers local derivative vectors from workers, exclude ghost cell
    comm.Gather( [d2px_loc[1:nx_loc+1], MPI.DOUBLE], [d2px, MPI.DOUBLE], root=0 )
    
    # --------------------------------------------------------
    # Time Extrapolation
    # --------------------------------------------------------
    pnew = c ** 2 * dt ** 2 * d2px + 2 * p - pold 
    # Add Source Term at isrc
    pnew[isrc] = pnew[isrc] + src[it] / (dx) * dt ** 2   # dx ??        
    # Remap Time Levels
    pold, p = p, pnew
    # Store solution in space-time on master 
    if rank == 0: P_xt.append(p)
    
comm.Barrier()    # Synchronize processors
t_final = (MPI.Wtime() - t_start)  # stop MPI timer

# Master prints time 
if (rank == 0): print('Computation time in seconds:   %s '% t_final)  
P_xt = np.asanyarray(P_xt)

### 3. Displaying the wave-field

In [ ]:
%%px
#---------------------------------------------------------------
# Master plots wavefield's numerical solution
#---------------------------------------------------------------
if rank == 0:  
    plt.figure(figsize=(6,8))
    plt.imshow(P_xt, cmap='coolwarm', aspect='auto', extent =[0, nx*dx, nt*dt, 0])
    plt.title('u(x,t) Acoustic Field')
    plt.ylabel('Time [s]')
    plt.xlabel('Space [m]')
    plt.show()
    #print('saving image ...')
    #plt.savefig('p_xt.png')    # Save your figure
    plt.close()

### 4. Animated wave-field

In [ ]:
%%px
#---------------------------------------------------------------
# Master animates wavefield solution
#---------------------------------------------------------------
if (rank == 0):
    fig = plt.figure(figsize=(10,5))
    ax = plt.axes(xlim=(0,nx*dx), ylim=(-0.0016,0.0016))
    line, = ax.plot([], [], lw=2, label='FDM')
    plt.title('1D Acoustic wave', fontsize=16)
    plt.ylabel('Amplitude', fontsize=12)
    plt.xlabel('x (m)', fontsize=12)
    plt.legend()
    
    # initialization function: plot the background of each frame
    def init():
        line.set_data([], [])
        return line,
    
    # animation function. This is called sequentially
    def animate(it):
        line.set_data(x, P_xt[it, :])
        return line,
    
    # call the animator.  blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=nt, interval=8, blit=True)
    #anim.save('p_xt.mp4', fps=30, extra_args=['-vcodec', 'libx264']) 
    plt.show()